In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, StandardScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss, mean_squared_error, f1_score, roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import label_binarize



In [16]:
csv_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
col_names = ['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width', 'Class']
iris_df = pd.read_csv(csv_url, header=None, names=col_names)


In [17]:
label_encoder = LabelEncoder()
iris_df['Class'] = label_encoder.fit_transform(iris_df['Class'])


In [18]:
X = iris_df[['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width']]
y = iris_df['Class']

In [19]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

In [20]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [21]:
### Logistic Regression with Gradient Descent (using LogisticRegression with 'saga' solver)
log_reg_pipeline = make_pipeline(StandardScaler(), LogisticRegression(solver='saga', max_iter=1000, multi_class='multinomial'))
log_loss_scores, rmse_scores, f1_scores, roc_auc_scores = [], [], [], []

for train_index, test_index in kf.split(X_poly):
    X_train, X_test = X_poly[train_index], X_poly[test_index]
    y_train, y_test = y[train_index], y[test_index]
    


In [22]:

log_reg_pipeline.fit(X_train, y_train)
    

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, multi_class='multinomial',
                                    solver='saga'))])

In [23]:
y_prob = log_reg_pipeline.predict_proba(X_test)
y_pred = log_reg_pipeline.predict(X_test)

In [24]:
log_reg_pipeline.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, multi_class='multinomial',
                                    solver='saga'))])

In [25]:
y_prob = log_reg_pipeline.predict_proba(X_test)
y_pred = log_reg_pipeline.predict(X_test)

In [26]:
log_loss_scores.append(log_loss(y_test, y_prob))
rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
f1_scores.append(f1_score(y_test, y_pred, average='weighted'))


In [27]:
roc_auc = roc_auc_score(label_binarize(y_test, classes=[0, 1, 2]), y_prob, average='weighted', multi_class='ovr')
roc_auc_scores.append(roc_auc)

In [28]:
# Output mean scores across folds for Logistic Regression (with Gradient Descent)
print("Logistic Regression (Gradient Descent) - K-Fold Cross-Validation Results:")
print("Mean Log Loss:", np.mean(log_loss_scores))
print("Mean RMSE:", np.mean(rmse_scores))
print("Mean F1 Score:", np.mean(f1_scores))
print("Mean ROC AUC:", np.mean(roc_auc_scores))

Logistic Regression (Gradient Descent) - K-Fold Cross-Validation Results:
Mean Log Loss: 0.08254486084578008
Mean RMSE: 0.0
Mean F1 Score: 1.0
Mean ROC AUC: 1.0


In [30]:
knn_model = KNeighborsClassifier(n_neighbors=5)
log_loss_scores_knn, rmse_scores_knn, f1_scores_knn, roc_auc_scores_knn = [], [], [], []

for train_index, test_index in kf.split(X_poly):
    X_train, X_test = X_poly[train_index], X_poly[test_index]
    y_train, y_test = y[train_index], y[test_index]
    knn_model.fit(X_train, y_train)


In [31]:
# Predict probabilities and classes
y_prob_knn = knn_model.predict_proba(X_test)
y_pred_knn = knn_model.predict(X_test)

In [32]:
log_loss_scores_knn.append(log_loss(y_test, y_prob_knn))
rmse_scores_knn.append(np.sqrt(mean_squared_error(y_test, y_pred_knn)))
f1_scores_knn.append(f1_score(y_test, y_pred_knn, average='weighted'))
    

In [33]:
roc_auc_knn = roc_auc_score(label_binarize(y_test, classes=[0, 1, 2]), y_prob_knn, average='weighted', multi_class='ovr')
roc_auc_scores_knn.append(roc_auc_knn)


In [34]:
print("\nK-Nearest Neighbors (K-Fold Cross-Validation) Results:")
print("Mean Log Loss:", np.mean(log_loss_scores_knn))
print("Mean RMSE:", np.mean(rmse_scores_knn))
print("Mean F1 Score:", np.mean(f1_scores_knn))
print("Mean ROC AUC:", np.mean(roc_auc_scores_knn))


K-Nearest Neighbors (K-Fold Cross-Validation) Results:
Mean Log Loss: 1.2163313497248527
Mean RMSE: 0.18257418583505536
Mean F1 Score: 0.9666666666666667
Mean ROC AUC: 0.9718810916179337
